In [1]:
import pandas as pd

# Укажите путь к файлу Excel
file_path = "ForScrapLinks.xlsx"

# Загрузите файл Excel в DataFrame, указав, что первая строка содержит заголовки столбцов
df_test = pd.read_excel(file_path, header=0)

In [ ]:
%%time
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Создайте DataFrame с вашими данными
df = df_test.iloc[4500:7085]

# Функция для выполнения парсинга и возврата новых данных
def parse_and_get_new_link(row):
    url = row["Links"]
    response = requests.get(url, verify=False)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"id": "search-result"})
        
        if table:
            links = []
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) > 1:
                    link = cells[1].find("a")
                    if link:
                        links.append(link.get("href"))
                        time.sleep(1)
            return " ".join(links)  # Возвращаем список ссылок в виде строки
            
        else:
            return "Таблица не найдена на странице."
            
    else:
        return "Не удалось получить страницу. Код ответа: " + str(response.status_code)

# Примените функцию к каждой строке DataFrame и сохраните результаты в новом столбце "LinksNEW"
df["LinksNEW"] = df.apply(parse_and_get_new_link, axis=1)

print(df)

In [3]:
df.to_csv("LINKS_GZ3.csv", index=False, sep=';')